!pip install -r ../app/requirements.txt

In [1]:
import urllib.request
import pandas as pd
import numpy as np

from pymongo import MongoClient
from mappings.geo import dict_state, dict_region
from mappings.ind import dict_sector, dict_industry
from mappings.occ import dict_occupation, dict_field
from mappings.edu import dict_schl, dict_educ

pd.options.display.float_format = '{:.4f}'.format

In [2]:
client = MongoClient('mongodb://0.0.0.0:27017/')

db = client.pums18

In [3]:
# Set API call  - SEX,MARHT,RACWHT,RACBLK,RACASN,
base = "https://api.census.gov"
dataset = "data/2018/acs/acs1/pums"
unfiltered = "SCHL,OCCP,INDP,REGION,ST,PUMA,PWGTP,ADJINC"
filtered = "COW=1:8&AGEP=20:80&WKHP=5:99&WAGP=500:99999"
url = f"{base}/{dataset}?get={unfiltered}&{filtered}"

In [4]:
# Get API data as text DataFrame
r = urllib.request.urlopen(url)
df0 = pd.read_json(r.read())
r.close()

In [5]:
# Get column names then drop first row
df0.columns = df0.iloc[0, :]
df0 = df0.drop(index=0)

In [6]:
# Cast all columns as numeric
for c in df0.columns :
    df0[c] = pd.to_numeric(df0[c])

In [7]:
# Define hourly wage based on 12mo avg 
df0['WAGE'] = (df0['WAGP']*df0['ADJINC'])/(df0['WKHP']*52)

In [8]:
# Calculate log of wage for modeling
df0['WLOG'] = np.log10(df0['WAGE'])

In [9]:
# Do updates on a copy of the original
df = df0.copy()

In [10]:
# Merge PUMA labels dataframe
df_pumas = pd.read_csv('../data/etc/PUMA_2010_Labels.csv')
df = df.merge(df_pumas, left_on=['ST','PUMA'], right_on=['STATEFP','PUMA5CE'], how='left')

In [11]:
# Rename columns
mapper = {
    'PUMA NAME':'LOCATION',
    'AGEP':'AGE',
    'PWGTP':'WGHT',
    'WAGP':'SALARY',
    'WKHP':'HOURS',
    'REGION':'REGION_CD'
}
df = df.rename(columns=mapper)

In [12]:
# Label coded values
df['SCHOOLING'] = df['SCHL'].map(dict_schl)
df['OCCUPATION'] = df['OCCP'].map(dict_occupation)
df['INDUSTRY'] = df['INDP'].map(dict_industry)
df['FIELD'] = df['OCCP'].map(dict_field)
df['SECTOR'] = df['INDP'].map(dict_sector)
df['STATE'] = df['ST'].map(dict_state)
df['REGION'] = df['REGION_CD'].map(dict_region)

In [13]:
# Add custom bins for age group and weekly hours
df['AGEGROUP'] = pd.cut(df['AGE'], [0,29,40,50,100],labels=['20s', '30s', '40s', '50+'])
df['WKHRS'] = pd.cut(df['HOURS'], [0,39,40,50,100],labels=['Under 40 hrs', '40 hrs', '41-50 hrs','Over 50 hrs'])

In [14]:
# Save data
df.to_csv('../data/etc/pums18_lab.csv', index=False)

In [17]:
locs = df[['STATE','LOCATION','ST','PUMA']].drop_duplicates().sort_values(['ST','PUMA']).reset_index(drop=True)
inds = df[['INDP','SECTOR','INDUSTRY']].drop_duplicates().sort_values(['INDP']).reset_index(drop=True)
occs = df[['OCCP','FIELD', 'OCCUPATION']].drop_duplicates().sort_values(['OCCP']).reset_index(drop=True)
educ = df[['SCHL','SCHOOLING']].drop_duplicates().sort_values(['SCHL']).reset_index(drop=True)

In [18]:
db.lab.drop()
db.loc.drop()
db.ind.drop()
db.occ.drop()
db.edu.drop()

db.lab.insert_many(df.to_dict('records'))
db.loc.insert_many(locs.to_dict('records'))
db.ind.insert_many(inds.to_dict('records'))
db.occ.insert_many(occs.to_dict('records'))
db.edu.insert_many(educ.to_dict('records'))